In [1]:
from madmex.overlay.extractions import zonal_stats_xarray
import numpy as np
import datacube
from datacube.api import GridWorkflow
from madmex.wrappers import gwf_query
from madmex.models import Tag
import fiona
import pprint
import os
from madmex.models import Model
from madmex.overlay.conversions import rasterize_xarray
from madmex.util import chunk
import xarray as xr
import gc
from affine import Affine
from rasterio.features import rasterize
from itertools import chain, islice
from collections import OrderedDict


In [2]:
name_of_product = 's2_20m_fiona_test_2_2018'
resolution = (-10,10)
tilesize = (50020,50020)
origin = (2426720, 977160)
proj4 = '+proj=lcc +lat_1=17.5 +lat_2=29.5 +lat_0=12 +lon_0=-102 +x_0=2500000 +y_0=0 +a=6378137 +b=6378136.027241431 +units=m +no_defs'


In [3]:
gwf_kwargs = {'region': 'Jalisco', 'begin': '2018-01-01', 'end':'2018-12-31',
              'product': name_of_product,'resolution': resolution, 'origin': origin,
              'proj4': proj4, 'tilesize': tilesize}

In [4]:
iterable = gwf_query(**gwf_kwargs)

In [5]:
lista_iter = list(iterable)

In [6]:
[lista_iter[x][0] for x in range(0,len(lista_iter))]

[(28, -27),
 (28, -28),
 (30, -29),
 (30, -30),
 (28, -29),
 (28, -30),
 (29, -30),
 (30, -27),
 (29, -29),
 (30, -28),
 (29, -28),
 (29, -27)]

In [7]:
var = lista_iter[8]

In [8]:
sr_0 = GridWorkflow.load(var[1],dask_chunks={'x': 2000, 'y': 2000, 'time': 1})

In [9]:
sr_0

<xarray.Dataset>
Dimensions:     (time: 1, x: 5002, y: 5002)
Coordinates:
  * time        (time) datetime64[ns] 2018-07-02
  * y           (y) float64 1.026e+06 1.026e+06 ... 9.762e+05 9.761e+05
  * x           (x) float64 2.428e+06 2.428e+06 ... 2.478e+06 2.478e+06
Data variables:
    blue_mean   (time, y, x) int16 dask.array<shape=(1, 5002, 5002), chunksize=(1, 2000, 2000)>
    green_mean  (time, y, x) int16 dask.array<shape=(1, 5002, 5002), chunksize=(1, 2000, 2000)>
    red_mean    (time, y, x) int16 dask.array<shape=(1, 5002, 5002), chunksize=(1, 2000, 2000)>
    re1_mean    (time, y, x) int16 dask.array<shape=(1, 5002, 5002), chunksize=(1, 2000, 2000)>
    re2_mean    (time, y, x) int16 dask.array<shape=(1, 5002, 5002), chunksize=(1, 2000, 2000)>
    re3_mean    (time, y, x) int16 dask.array<shape=(1, 5002, 5002), chunksize=(1, 2000, 2000)>
    nir_mean    (time, y, x) int16 dask.array<shape=(1, 5002, 5002), chunksize=(1, 2000, 2000)>
    swir1_mean  (time, y, x) int16 dask.array

In [10]:
s = fiona.open('s3://mysegmentation.shp')

In [11]:
geom_list = [x['geometry'] for x in s]


In [12]:
geom_list[1000]

{'type': 'Polygon',
 'coordinates': [[(2438580.0, 1021170.0),
   (2438620.0, 1021170.0),
   (2438620.0, 1021160.0),
   (2438660.0, 1021160.0),
   (2438660.0, 1021170.0),
   (2438740.0, 1021170.0),
   (2438740.0, 1021160.0),
   (2438750.0, 1021160.0),
   (2438750.0, 1020650.0),
   (2438740.0, 1020650.0),
   (2438250.0, 1020650.0),
   (2438250.0, 1021160.0),
   (2438360.0, 1021160.0),
   (2438360.0, 1021150.0),
   (2438400.0, 1021150.0),
   (2438400.0, 1021160.0),
   (2438500.0, 1021160.0),
   (2438500.0, 1021150.0),
   (2438520.0, 1021150.0),
   (2438520.0, 1021160.0),
   (2438530.0, 1021160.0),
   (2438530.0, 1021150.0),
   (2438540.0, 1021150.0),
   (2438540.0, 1021160.0),
   (2438570.0, 1021160.0),
   (2438580.0, 1021160.0),
   (2438580.0, 1021170.0)]]}

In [13]:
len(geom_list)

10000

In [14]:
aff = Affine(*list(sr_0.affine)[0:6])

In [15]:
dimensions_dataset = list(sr_0.coords)
list_dimensions = [x for x in dimensions_dataset if x != 'time']
lambda_function = lambda l_netcdf,l_test: l_netcdf[0] if l_netcdf[0] in l_test else l_netcdf[1]
xdim = lambda_function(list_dimensions,['x','longitude'])
ydim = lambda_function(list_dimensions,['y','latitude'])

In [16]:
iterable_zip = zip(geom_list, range(1, len(geom_list) + 1))

In [56]:
aff

Affine(10.0, 0.0, 2427740.0,
       0.0, -10.0, 1026160.0)

In [17]:
fc_raster = rasterize(iterable_zip, transform=aff,
                          out_shape=(sr_0.sizes[ydim], sr_0.sizes[xdim]),
                      dtype='float64', fill=np.nan)


In [18]:
fc_raster


array([[1.000e+00, 1.000e+00, 1.000e+00, ..., 8.100e+01, 8.100e+01,
        8.100e+01],
       [1.000e+00, 1.000e+00, 1.000e+00, ..., 8.100e+01, 8.100e+01,
        8.100e+01],
       [1.000e+00, 1.000e+00, 1.000e+00, ..., 8.100e+01, 8.100e+01,
        8.100e+01],
       ...,
       [9.901e+03, 9.901e+03, 9.901e+03, ..., 9.992e+03, 9.992e+03,
        9.992e+03],
       [9.901e+03, 9.901e+03, 9.901e+03, ..., 9.992e+03, 9.992e+03,
        9.992e+03],
       [9.901e+03, 9.901e+03, 9.901e+03, ..., 9.992e+03, 9.992e+03,
        9.992e+03]])

In [37]:
fc_raster[0]

array([ 1.,  1.,  1., ..., 81., 81., 81.])

In [38]:
fc_raster.shape

(5002, 5002)

In [39]:
fc_raster[1]

array([ 1.,  1.,  1., ..., 81., 81., 81.])

In [53]:
fc_raster[50]

array([ 1.,  1.,  1., ..., 81., 81., 81.])

In [52]:
fc_raster[51]

array([108., 108., 108., ..., 171., 171., 171.])

In [55]:
fc_raster[70]

array([108., 108., 108., ..., 171., 171., 171.])

In [54]:
fc_raster[101]

array([204., 204., 204., ..., 277., 277., 277.])

In [45]:
fc_raster[5000]

array([9901., 9901., 9901., ..., 9992., 9992., 9992.])

In [44]:
fc_raster[5001]

array([9901., 9901., 9901., ..., 9992., 9992., 9992.])

In [19]:
xr_arr = xr.DataArray(fc_raster, dims=[ydim, xdim], name='features_id')

In [20]:
xr_arr

<xarray.DataArray 'features_id' (y: 5002, x: 5002)>
array([[1.000e+00, 1.000e+00, 1.000e+00, ..., 8.100e+01, 8.100e+01, 8.100e+01],
       [1.000e+00, 1.000e+00, 1.000e+00, ..., 8.100e+01, 8.100e+01, 8.100e+01],
       [1.000e+00, 1.000e+00, 1.000e+00, ..., 8.100e+01, 8.100e+01, 8.100e+01],
       ...,
       [9.901e+03, 9.901e+03, 9.901e+03, ..., 9.992e+03, 9.992e+03, 9.992e+03],
       [9.901e+03, 9.901e+03, 9.901e+03, ..., 9.992e+03, 9.992e+03, 9.992e+03],
       [9.901e+03, 9.901e+03, 9.901e+03, ..., 9.992e+03, 9.992e+03, 9.992e+03]])
Dimensions without coordinates: y, x

In [21]:
combined = xr.merge([xr_arr, sr_0])

In [22]:
combined

<xarray.Dataset>
Dimensions:      (time: 1, x: 5002, y: 5002)
Coordinates:
  * x            (x) float64 2.428e+06 2.428e+06 ... 2.478e+06 2.478e+06
  * y            (y) float64 1.026e+06 1.026e+06 ... 9.762e+05 9.761e+05
  * time         (time) datetime64[ns] 2018-07-02
Data variables:
    features_id  (y, x) float64 1.0 1.0 1.0 ... 9.992e+03 9.992e+03 9.992e+03
    blue_mean    (time, y, x) int16 dask.array<shape=(1, 5002, 5002), chunksize=(1, 2000, 2000)>
    green_mean   (time, y, x) int16 dask.array<shape=(1, 5002, 5002), chunksize=(1, 2000, 2000)>
    red_mean     (time, y, x) int16 dask.array<shape=(1, 5002, 5002), chunksize=(1, 2000, 2000)>
    re1_mean     (time, y, x) int16 dask.array<shape=(1, 5002, 5002), chunksize=(1, 2000, 2000)>
    re2_mean     (time, y, x) int16 dask.array<shape=(1, 5002, 5002), chunksize=(1, 2000, 2000)>
    re3_mean     (time, y, x) int16 dask.array<shape=(1, 5002, 5002), chunksize=(1, 2000, 2000)>
    nir_mean     (time, y, x) int16 dask.array<shape=

In [23]:
combined = combined.stack(z=(xdim, ydim))

In [24]:
combined

<xarray.Dataset>
Dimensions:      (time: 1, z: 25020004)
Coordinates:
  * time         (time) datetime64[ns] 2018-07-02
  * z            (z) MultiIndex
  - x            (z) float64 2.428e+06 2.428e+06 ... 2.428e+06 2.428e+06
  - y            (z) float64 1.026e+06 1.026e+06 ... 1.026e+06 1.026e+06
Data variables:
    features_id  (z) float64 1.0 1.0 1.0 1.0 ... 9.992e+03 9.992e+03 9.992e+03
    blue_mean    (time, z) int16 dask.array<shape=(1, 25020004), chunksize=(1, 3331332)>
    green_mean   (time, z) int16 dask.array<shape=(1, 25020004), chunksize=(1, 3331332)>
    red_mean     (time, z) int16 dask.array<shape=(1, 25020004), chunksize=(1, 3331332)>
    re1_mean     (time, z) int16 dask.array<shape=(1, 25020004), chunksize=(1, 3331332)>
    re2_mean     (time, z) int16 dask.array<shape=(1, 25020004), chunksize=(1, 3331332)>
    re3_mean     (time, z) int16 dask.array<shape=(1, 25020004), chunksize=(1, 3331332)>
    nir_mean     (time, z) int16 dask.array<shape=(1, 25020004), chunksiz

In [25]:
combined=combined.reset_index('z')

In [26]:
combined

<xarray.Dataset>
Dimensions:      (time: 1, z: 25020004)
Coordinates:
  * time         (time) datetime64[ns] 2018-07-02
    x            (z) float64 2.428e+06 2.428e+06 ... 2.478e+06 2.478e+06
    y            (z) float64 1.026e+06 1.026e+06 ... 9.762e+05 9.761e+05
Dimensions without coordinates: z
Data variables:
    features_id  (z) float64 1.0 1.0 1.0 1.0 ... 9.992e+03 9.992e+03 9.992e+03
    blue_mean    (time, z) int16 dask.array<shape=(1, 25020004), chunksize=(1, 3331332)>
    green_mean   (time, z) int16 dask.array<shape=(1, 25020004), chunksize=(1, 3331332)>
    red_mean     (time, z) int16 dask.array<shape=(1, 25020004), chunksize=(1, 3331332)>
    re1_mean     (time, z) int16 dask.array<shape=(1, 25020004), chunksize=(1, 3331332)>
    re2_mean     (time, z) int16 dask.array<shape=(1, 25020004), chunksize=(1, 3331332)>
    re3_mean     (time, z) int16 dask.array<shape=(1, 25020004), chunksize=(1, 3331332)>
    nir_mean     (time, z) int16 dask.array<shape=(1, 25020004), chunks

In [27]:
combined=combined.drop([xdim,ydim])

In [28]:
combined

<xarray.Dataset>
Dimensions:      (time: 1, z: 25020004)
Coordinates:
  * time         (time) datetime64[ns] 2018-07-02
Dimensions without coordinates: z
Data variables:
    features_id  (z) float64 1.0 1.0 1.0 1.0 ... 9.992e+03 9.992e+03 9.992e+03
    blue_mean    (time, z) int16 dask.array<shape=(1, 25020004), chunksize=(1, 3331332)>
    green_mean   (time, z) int16 dask.array<shape=(1, 25020004), chunksize=(1, 3331332)>
    red_mean     (time, z) int16 dask.array<shape=(1, 25020004), chunksize=(1, 3331332)>
    re1_mean     (time, z) int16 dask.array<shape=(1, 25020004), chunksize=(1, 3331332)>
    re2_mean     (time, z) int16 dask.array<shape=(1, 25020004), chunksize=(1, 3331332)>
    re3_mean     (time, z) int16 dask.array<shape=(1, 25020004), chunksize=(1, 3331332)>
    nir_mean     (time, z) int16 dask.array<shape=(1, 25020004), chunksize=(1, 3331332)>
    swir1_mean   (time, z) int16 dask.array<shape=(1, 25020004), chunksize=(1, 3331332)>
    swir2_mean   (time, z) int16 dask.a

In [42]:
#combined = combined.stack(z=(xdim, ydim)).reset_index('z').drop([xdim, ydim])

In [43]:
combined

<xarray.Dataset>
Dimensions:      (time: 1, z: 25020004)
Coordinates:
  * time         (time) datetime64[ns] 2018-07-02
Dimensions without coordinates: z
Data variables:
    features_id  (z) float64 1.0 1.0 1.0 1.0 ... 9.992e+03 9.992e+03 9.992e+03
    blue_mean    (time, z) int16 dask.array<shape=(1, 25020004), chunksize=(1, 3331332)>
    green_mean   (time, z) int16 dask.array<shape=(1, 25020004), chunksize=(1, 3331332)>
    red_mean     (time, z) int16 dask.array<shape=(1, 25020004), chunksize=(1, 3331332)>
    re1_mean     (time, z) int16 dask.array<shape=(1, 25020004), chunksize=(1, 3331332)>
    re2_mean     (time, z) int16 dask.array<shape=(1, 25020004), chunksize=(1, 3331332)>
    re3_mean     (time, z) int16 dask.array<shape=(1, 25020004), chunksize=(1, 3331332)>
    nir_mean     (time, z) int16 dask.array<shape=(1, 25020004), chunksize=(1, 3331332)>
    swir1_mean   (time, z) int16 dask.array<shape=(1, 25020004), chunksize=(1, 3331332)>
    swir2_mean   (time, z) int16 dask.a

In [29]:
combined = combined.where(np.isfinite(combined['features_id']), drop=True)

In [30]:
combined

<xarray.Dataset>
Dimensions:      (time: 1, z: 25020004)
Coordinates:
  * time         (time) datetime64[ns] 2018-07-02
Dimensions without coordinates: z
Data variables:
    features_id  (z) float64 1.0 1.0 1.0 1.0 ... 9.992e+03 9.992e+03 9.992e+03
    blue_mean    (time, z) float64 dask.array<shape=(1, 25020004), chunksize=(1, 3331332)>
    green_mean   (time, z) float64 dask.array<shape=(1, 25020004), chunksize=(1, 3331332)>
    red_mean     (time, z) float64 dask.array<shape=(1, 25020004), chunksize=(1, 3331332)>
    re1_mean     (time, z) float64 dask.array<shape=(1, 25020004), chunksize=(1, 3331332)>
    re2_mean     (time, z) float64 dask.array<shape=(1, 25020004), chunksize=(1, 3331332)>
    re3_mean     (time, z) float64 dask.array<shape=(1, 25020004), chunksize=(1, 3331332)>
    nir_mean     (time, z) float64 dask.array<shape=(1, 25020004), chunksize=(1, 3331332)>
    swir1_mean   (time, z) float64 dask.array<shape=(1, 25020004), chunksize=(1, 3331332)>
    swir2_mean   (time,

In [31]:
df = combined.to_dataframe()

In [32]:
df

features_id  blue_mean  green_mean  red_mean  re1_mean  \
time       z                                                                  
2018-07-02 0                 1.0      607.0       832.0    1106.0    1299.0   
           1                 1.0      607.0       832.0    1106.0    1299.0   
           2                 1.0      531.0       784.0     972.0    1160.0   
           3                 1.0      531.0       784.0     972.0    1160.0   
           4                 1.0      578.0       830.0    1025.0    1293.0   
           5                 1.0      578.0       830.0    1025.0    1293.0   
           6                 1.0      605.0       861.0    1101.0    1361.0   
           7                 1.0      605.0       861.0    1101.0    1361.0   
           8                 1.0      574.0       806.0    1052.0    1271.0   
           9                 1.0      574.0       806.0    1052.0    1271.0   
           10                1.0      454.0       632.0     823.0    1053.0   
           11                1.0      454.0       632.0     823.0    1053.0   
           12                1.0      461.0       592.0     783.0     916.0   
           13                1.0      461.0       592.0     783.0     916.0   
           14                1.0      436.0       586.0     756.0     852.0   
           15                1.0      436.0       586.0     756.0     852.0   
           16                1.0      399.0       511.0     689.0     800.0   
           17                1.0      399.0       511.0     689.0     800.0   
           18                1.0      451.0       587.0     773.0     870.0   
           19                1.0      451.0       587.0     773.0     870.0   
           20                1.0      527.0       687.0     884.0    1064.0   
           21                1.0      527.0       687.0     884.0    1064.0   
           22                1.0      578.0       742.0    1001.0    1205.0   
           23                1.0      578.0       742.0    1001.0    1205.0   
           24                1.0      548.0       769.0    1076.0    1270.0   
           25                1.0      548.0       769.0    1076.0    1270.0   
           26                1.0      563.0       742.0     947.0    1186.0   
           27                1.0      563.0       742.0     947.0    1186.0   
           28                1.0      721.0       944.0    1264.0    1386.0   
           29                1.0      721.0       944.0    1264.0    1386.0   
...                          ...        ...         ...       ...       ...   
           25019974       9992.0      315.0       529.0     668.0    1017.0   
           25019975       9992.0      315.0       529.0     668.0    1017.0   
           25019976       9992.0      307.0       528.0     632.0     915.0   
           25019977       9992.0      307.0       528.0     632.0     915.0   
           25019978       9992.0      472.0       723.0     944.0    1220.0   
           25019979       9992.0      472.0       723.0     944.0    1220.0   
           25019980       9992.0      565.0       831.0    1135.0    1352.0   
           25019981       9992.0      565.0       831.0    1135.0    1352.0   
           25019982       9992.0      492.0       739.0     989.0    1290.0   
           25019983       9992.0      492.0       739.0     989.0    1290.0   
           25019984       9992.0      521.0       766.0    1024.0    1227.0   
           25019985       9992.0      521.0       766.0    1024.0    1227.0   
           25019986       9992.0      389.0       626.0     803.0    1180.0   
           25019987       9992.0      389.0       626.0     803.0    1180.0   
           25019988       9992.0      401.0       629.0     831.0    1008.0   
           25019989       9992.0      401.0       629.0     831.0    1008.0   
           25019990       9992.0      286.0       495.0     609.0     930.0   
           25019991       9992.0      286.0       495.0     609.0     9

In [34]:
var_list = list(sr_0.data_vars)
categorical_variables = []
agg_list = [(k, 'mean') if k not in categorical_variables else (k, 'first') for k in var_list]
agg_ordered_dict = OrderedDict(agg_list)

In [35]:
df = df.groupby('features_id').agg(agg_ordered_dict)

In [36]:
df

,blue_mean,green_mean,red_mean,re1_mean,re2_mean,re3_mean,nir_mean,swir1_mean,swir2_mean,ndvi_mean,ndmi_mean,ndvi_max,ndvi_min,ndmi_max,ndmi_min,elevation,slope,aspect
features_id,,,,,,,,,,,,,,,,,,
1.0,472.582853,654.138793,840.590542,1026.809304,1171.425606,1272.611688,1435.298731,1960.171088,1478.199923,2706.951173,-1551.542868,2940.077278,2473.809304,-1348.642445,-1754.384468,1628.316032,4.978980,176.500183
2.0,453.378039,639.167451,823.304706,1025.450588,1208.492157,1326.393725,1511.043529,2093.258039,1540.616078,3086.632157,-1557.924706,3278.047059,2895.247451,-1363.750196,-1752.105490,1648.298039,10.803452,278.774841
3.0,518.131373,728.714510,973.007451,1181.454118,1362.098039,1490.629412,1700.469412,2509.421176,1821.294118,2918.351373,-1875.880392,3054.944314,2781.790196,-1640.109804,-2111.705098,1666.312549,9.250080,199.561813
4.0,370.210588,526.901569,688.634118,862.292549,1035.648235,1147.294118,1326.683922,2094.444706,1477.023529,3236.949804,-2192.955294,3402.034118,3071.890196,-1987.926275,-2398.019216,1656.573725,4.998461,84.255821
5.0,488.187843,672.152549,856.782353,1061.650588,1261.118039,1384.184706,1576.241961,2219.468235,1594.370980,3138.469412,-1668.142353,3267.771765,3009.160000,-1483.856078,-1852.416863,1648.311373,5.208080,234.216232
6.0,453.423922,623.374510,764.794118,977.790588,1216.125098,1341.350980,1532.940392,2145.320784,1510.585490,3445.531373,-1654.125098,3593.034510,3298.008235,-1464.071373,-1844.190196,1686.163137,6.386632,236.883743
7.0,480.037647,660.080000,857.830980,1068.420784,1259.284314,1388.633725,1601.849412,2311.737647,1634.026275,3086.871765,-1823.048235,3214.714902,2959.058824,-1657.716471,-1988.338431,1715.738824,11.741992,201.034744
8.0,425.009412,585.106275,748.577255,951.054902,1148.786275,1273.464314,1473.609020,2193.255686,1568.359608,3348.086667,-1930.550588,3491.250980,3204.922745,-1770.880000,-2090.209412,1730.147843,11.204632,188.411377
9.0,443.653333,607.083137,718.105098,948.784314,1235.028235,1367.572941,1568.856863,2044.037255,1469.433333,3826.972549,-1306.035686,4006.374902,3647.537647,-1144.971373,-1467.098039,1739.509804,10.549579,212.592239


In [57]:
ids = list(df.index.values.astype('uint32') - 1)

In [67]:
len(ids)

10000

In [76]:
y=np.array([s[x]['properties']['id'] for x in range(0,len(ids))])

In [77]:
y

array([   0,    1,    2, ..., 9979, 9904, 9969])

In [78]:
y.shape

(10000,)

In [66]:
s[1]['properties']['id']

1

In [79]:
s.close()

In [32]:
ids_list = [x['id'] for x in s]

In [34]:
len(ids_list)

10000

In [35]:
iterable_zip2 = zip(geom_list, ids_list)

In [37]:
fc_raster = rasterize(iterable_zip2, transform=aff,
                          out_shape=(sr_0.sizes[ydim], sr_0.sizes[xdim]),
                      dtype='float64', fill=np.nan)

TypeError: cannot perform reduce with flexible type